I200635
Mahnoor Akhtar
Section A CS

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
os.chdir('/content/drive/MyDrive/TEXT_TO_SPEECH')

In [ ]:
import pandas as pd

# Load the CSV file into a DataFrame
df = pd.read_csv("output_final.csv")

# Create empty lists to store recording paths and transcripts
recording_paths = []
transcripts = []

# Iterate through the rows of the DataFrame
for _, row in df.iterrows():
    # Append the path to the "Recording" folder and ".mp3" extension
    recording_path = f"Recording/{row['original_sentence_id']}.mp3"
    # Append the transcript
    transcript = row['sentence']

    recording_paths.append(recording_path)
    transcripts.append(transcript)

# Check the resulting lists
print("Recording Paths:")
print(recording_paths[:5])  # Print the first 5 paths
print("\nTranscripts:")
print(transcripts[:5])  # Print the first 5 transcripts


Recording Paths:
['Recording/00033179f81985c4a0d8450b6a080cd31f28c194a689e436206e50dd6459f573.mp3', 'Recording/0009742ca8d7e3d4181d8f9192a0a00972f4198284cf4ee044df2f295680c50f.mp3', 'Recording/0009ceeaf3b70994b4783de95cc760f3f666b3436505c0e0df4f5e84c18e987a.mp3', 'Recording/001aec9891863e28984b7171204d23397425a2f69bdf09fd9af18fb8f677748b.mp3', 'Recording/00237dae665880df42aec4615fdae5c3a82de5ea6526ae2cf3c97258f86c7567.mp3']

Transcripts:
['ان میں سے ایک یہ قانون ہے۔', 'پیٹرول کی قیمت  روپے ڈیزل  پیسے فی لیٹر کم کرنے کا اعلان', 'محو حیرت ہوں کہ دنیا کیا سے کیا ہو جائے گی', 'اس کا ایک بڑا سبب ان اداروں میں حکومتی مداخلت ہے۔', 'وہ اسے شیعہ سنی رنگ دے رہے ہیں۔']


In [ ]:
for recording_path in recording_paths:
    recording_folder_path = os.path.join("Recording", os.path.basename(recording_path))
    if not os.path.exists(recording_folder_path):
        print("The recording at path {} is not in the Recording folder.".format(recording_path))
    else:
        recording_folder_size = os.path.getsize(recording_folder_path)
        recording_size = os.path.getsize(recording_path)
        if recording_folder_size != recording_size:
            print("The recording at path {} is not the same size as the corresponding recording in the Recording folder.".format(recording_path))


In [ ]:
!pip install accelerator -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 8.7 MB/s eta 0:00:00


In [ ]:

!pip install transformers
!pip install torchaudio
!pip install datasets
!pip install torch torchaudio transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 72.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 36.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 493.7/493.7 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.8 MB/s eta 0:00:00


In [ ]:
import os
import torch
from transformers import Trainer, TrainingArguments
from datasets import load_dataset
import torchaudio
from torchaudio.transforms import Resample
import pandas as pd
from transformers import Wav2Vec2ForCTC, Wav2Vec2Processor,AutoTokenizer
from torch.utils.data import Dataset

In [ ]:
import json

custom_vocab_path = "/content/drive/MyDrive/TEXT_TO_SPEECH/vocab.json"

with open(custom_vocab_path, "r", encoding="utf-8") as json_file:
    custom_vocab = json.load(json_file)

if ' ' not in custom_vocab:
    custom_vocab[' '] = len(custom_vocab)

# Check and add the [PAD] token if it doesn't exist
if "[PAD]" not in custom_vocab:
    custom_vocab["[PAD]"] = len(custom_vocab)

In [ ]:
from transformers import AutoTokenizer, AutoModelForCTC
from transformers import Wav2Vec2Processor

# Initialize the Whisper tokenizer and model
model_name = "facebook/wav2vec2-base-960h"
tokenizer = AutoTokenizer.from_pretrained(model_name, vocab=custom_vocab, unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")
model = AutoModelForCTC.from_pretrained(model_name)
processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-base-960h")

Some weights of the model checkpoint at facebook/wav2vec2-base-960h were not used when initializing Wav2Vec2ForCTC: ['wav2vec2.encoder.pos_conv_embed.conv.weight_v', 'wav2vec2.encoder.pos_conv_embed.conv.weight_g']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'wav2vec2.encoder.pos_conv_embed.conv.parametrizations.weight.original0', 'wav2vec2.masked_spec_embed']
You sho

In [ ]:

import torchaudio
import torch

class CustomFeatureExtractor(object):
    def __init__(self, num_mel_bins, max_audio_length):
        self.num_mel_bins = num_mel_bins
        self.max_audio_length = max_audio_length

    def extract_features(self, waveform):

        # Extract features from the waveform
        mel_spectrogram = torchaudio.transforms.MelSpectrogram(
            sample_rate=16000,  # Adjust the sample rate as needed
            n_fft=400,  # Adjust the window size as needed
            hop_length=self.max_audio_length // self.num_mel_bins,  # Adjust the hop length to fit the desired dimensions
            n_mels=self.num_mel_bins,  # Use the specified number of mel bins
            center=False,  # Make sure the window is not centered
        )(waveform)

        # Crop or pad to ensure consistent size
        if mel_spectrogram.shape[1] > self.max_audio_length // self.num_mel_bins:
            mel_spectrogram = mel_spectrogram[:, :self.max_audio_length // self.num_mel_bins]
        else:
            padding = self.max_audio_length // self.num_mel_bins - mel_spectrogram.shape[1]
            mel_spectrogram = F.pad(mel_spectrogram, (0, padding))

        # Return the features as a Tensor object
        return mel_spectrogram


In [ ]:
from torch.nn import functional as F
from torch.nn.utils.rnn import pad_sequence  # Import pad_sequence

class CustomDataset(Dataset):
    def __init__(self, recording_paths, transcripts, tokenizer, feature_extractor, max_audio_length, sampling_rate=16000):
        self.recording_paths = recording_paths
        self.transcripts = transcripts
        self.tokenizer = tokenizer
        self.feature_extractor = feature_extractor
        self.max_audio_length = max_audio_length

    def __len__(self):
        return len(self.recording_paths)

    def __getitem__(self, idx):
        path = self.recording_paths[idx]

        transcript = self.transcripts[idx]

        # Load audio and normalize
        waveform, _ = torchaudio.load(path, normalize=True)

        # Ensure that waveform is a PyTorch tensor
        if not isinstance(waveform, torch.Tensor):
            waveform = torch.from_numpy(waveform)

        print("Waveform shape:", waveform.shape)
        # Apply padding or truncation to ensure uniform audio length
        if waveform.shape[1] > self.max_audio_length:
            waveform = waveform[:, :self.max_audio_length]
        else:
            padding = self.max_audio_length - waveform.shape[1]
            waveform = F.pad(waveform, (0, padding))


        print("waveform shape after truncation:", waveform.shape)

        # Extract features using your custom feature extractor
        input_values = self.feature_extractor.extract_features(waveform)

        # Tokenize the transcript using your custom tokenizer
        transcript_tokens = self.tokenizer.tokenize(transcript)
        input_ids = self.tokenizer.convert_tokens_to_ids(transcript_tokens)

        SPECIAL_TOKEN = "[ISSUE]"

        # Get the token ID for the special token
        SPECIAL_TOKEN_ID = tokenizer.convert_tokens_to_ids(SPECIAL_TOKEN)

        if not input_ids:
        # You can use a special token to indicate an issue, or simply add a placeholder token.
            input_ids = [SPECIAL_TOKEN_ID]

        print("Input values shape:", input_values.shape)
        print("Input IDs length:", len(input_ids))

        # Return the input values and input IDs
        return {
            "input_values": input_values,
            "input_ids": input_ids,

        }




In [ ]:
def convert_to_tensor(waveform):
  if isinstance(waveform, int):
    waveform = torch.from_numpy(waveform)
  return waveform

In [ ]:
import torch
from torch.utils.data import DataLoader

In [ ]:
num_mel_bins = 40  # Adjust as needed
hop_length = 160  # Adjust as needed
feature_extractor = CustomFeatureExtractor(num_mel_bins, hop_length)

In [ ]:
import torch
from torch.utils.data import DataLoader

max_audio_lenght = 20000  # You can adjust this value as needed
dataset=CustomDataset(recording_paths, transcripts, tokenizer, feature_extractor, max_audio_lenght)
# Create a data loader

dataloader = DataLoader(
    dataset,
    batch_size=4,
    shuffle=True,


)
# Fine-tune the speech model
model.train()
for epoch in range(10):
    for batch in dataloader:

        # Check if the waveform variable is an integer
        batch["input_values"] = convert_to_tensor(batch["input_values"])

        # Convert the input_values to a Tensor object
        input_values = batch["input_values"]

        # Extract the input_ids
        input_ids = batch["input_ids"]

        # Forward pass
        outputs = model(input_values)

        # Loss calculation
        loss = outputs.loss

        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

# Save the fine-tuned speech model
torch.save(model.state_dict(), "fine-tuned-speech-model.pt")


Waveform shape: torch.Size([1, 138240])
waveform shape after truncation: torch.Size([1, 20000])
Input values shape: torch.Size([1, 4, 4901])
Input IDs length: 35
Waveform shape: torch.Size([1, 167040])
waveform shape after truncation: torch.Size([1, 20000])
Input values shape: torch.Size([1, 4, 4901])
Input IDs length: 32
Waveform shape: torch.Size([1, 144000])
waveform shape after truncation: torch.Size([1, 20000])
Input values shape: torch.Size([1, 4, 4901])
Input IDs length: 48
Waveform shape: torch.Size([1, 161280])
waveform shape after truncation: torch.Size([1, 20000])
Input values shape: torch.Size([1, 4, 4901])
Input IDs length: 33


RuntimeError: ignored

In [ ]:

for i in dataset:
  # if(i==None):
  #       print("Found")
  #       break
    print(i)

Streaming output truncated to the last 5000 lines.
Waveform shape: torch.Size([1, 150912])
waveform shape after truncation: torch.Size([1, 20000])
Input values shape: torch.Size([1, 4, 4901])
Input IDs length: 34
{'input_values': tensor([[[1.5088e-13, 1.5391e-13, 1.5563e-13,  ..., 3.8284e+00,
          3.7583e+00, 3.6895e+00],
         [2.6416e-13, 2.9395e-13, 3.2583e-13,  ..., 1.8445e+01,
          1.8349e+01, 1.8259e+01],
         [3.6531e-13, 4.3188e-13, 5.0723e-13,  ..., 1.1286e+01,
          1.1346e+01, 1.1395e+01],
         [3.1270e-13, 3.7333e-13, 4.4310e-13,  ..., 5.6209e-01,
          5.7659e-01, 5.8996e-01]]]), 'input_ids': [None, None, None, None, 4, None, None, None, None, None, 4, None, None, 4, None, None, None, 4, None, None, None, None, 4, None, None, 4, None, None, None, 4, None, None, None, None], 'input_ids_length': 34}
Waveform shape: torch.Size([1, 185472])
waveform shape after truncation: torch.Size([1, 20000])
Input values shape: torch.Size([1, 4, 4901])
Input ID